# Pyomo installation

In [1]:
!pip install -q pyomo


In [40]:
!apt-get install -y -qq glpk-utils > /dev/null 2>&1

# Problema de mezcla lineal
Como ingeniero de procesos de una cervecería, una de tus responsabilidades se basa en planificar la producción.

Habeís recibido un pedido de 100 galones de cerveza, cuya graduación debe ser del 4% de alcohol por volumen. La cervecería tiene dos líneas de producción. La cerveza A tiene un porcentaje de alcohol del 4.5% y un coste de producción 0.32 USD/galón mientras que la cerveza B de 3.7% tiene unos costes de producción 0.25 USD/galón. Como alternativa, se puede utilizar agua como agente de mezcla para conseguir la graduación requerida por el cliente, a un coste de 0.05 USD/galón.

Tu tarea es encontrar la mezcla de coste mínimo que cumpla las especificaciones del cliente.

## Datos
El primer paso consiste en represntar los datos del problema como un diccionario. Una de las principales ventajas de elegir esta estructura es que podría extenderse si hiciera falta, por ejemplo, para añadir posibles componenetes adicionales.

In [ ]:
data =

## Implementación del modelo
El primer paso consiste en importar Pyomo y definir el tipo de modelo que se va a utilizar.

In [6]:
import pyomo.environ as pyo

m  = pyo.ConcreteModel()

### Sets

In [7]:
m.C = pyo.Set( initialize = ['A','B','C'], doc = 'Components')

### Variables

In [15]:
m.x = pyo.Var(m.C, within=pyo.NonNegativeReals , doc = 'Volumen m3')

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


## Parámetros

In [17]:
m.P = pyo.Param(m.C, initialize = {'A': 0.32, 'B': 0.25, 'C': 0.05}, doc = 'Coste €/m3')
m.apv = pyo.Param( m.C, initialize = {'A': 4.5, 'B': 3.7, 'C': 0.}, doc = 'Graduación')
m.V = pyo.Param(initialize = 100, doc = 'Demanda m3')
m.apv_spec = pyo.Param(initialize = 4., doc = 'Graduación deseada')

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


### Funcion objetivo
$$
\begin{align}
\text{coste} & = \sum_{c\in C} x_c P_c \nonumber
\end{align}
$$
Para definir la función objetivo, se utiliza el comando `Objective` cuya sintaxis es la siguiente:
`Objective(expr=myfunction, sense=keyword)`
- expr: función que devuelve un único valor
- sense: `maximize` o `minimize`


In [18]:
m.obj = pyo.Objective(expr = sum(m.x[c]*m.P[c]for c in m.C), sense=pyo.minimize)

### Restricciones
De igual manera, el comando `Constraint(expr=myfunction)`sirve para definir las restricciones del problema. En este caso, el comando solo toma un argumento: la función con la que evaluar cada una de las restricciones.
- Satisfacción de la demanda
$
\begin{align}
V &  = \sum_{c\in C} x_c \nonumber
\end{align}
$

In [20]:
m.con_demand = pyo.Constraint(expr = m.V == sum(m.x[c] for c in m.C))

- Composición del producto
$$
\begin{align}
\bar{A} & = \frac{\sum_{c\in C}x_c A_c}{\sum_{c\in C} x_c} & \text{¡No es lineal!} \nonumber
\end{align}
$$
$$
\begin{align}
0 & = \sum_{c\in C}x_c \left(A_c - \bar{A}\right) & \text{} \nonumber
\end{align}
$$

In [23]:
m.con_spec = pyo.Constraint(expr = 0 == sum(m.x[c]*(m.apv[c] - m.apv_spec) for c in m.C))

## Selección y llamada al solver
El algoritmo que se va a utilizar en la resolución del problema se llama con la orden `SolverFactory(solver_name)`, donde solver `solver_name` debe ser un dato de tipo cadena (string). En este caso, se va a utilizar el paquete GLPK (GNU Linear Programming Kit).

In [26]:
optimizer = pyo.SolverFactory('glpk', executable='/usr/bin/glpsol')
results = optimizer.solve(m)

## Inspeccionar la solución
Una vez ejecutado el comando solve, nuestro objeto `model` se ha modificado y el valor de las variables se corresponde con el de la solución óptima. Para visualizar qué valor ha tomado cada variable hay dos opciones:
- Utilizar el comando `pprint()`. La sintaxis correcta es `model.variable.pprint()`, aunque también se puede mostrar toda la información de una vez escribiendo `model.pprint()`.
- Accediendo directamente a la solución por medio de `model.variable()`.

In [33]:
print('Optimal Blend')
for c in m.C:
    print('  ', c, ':', m.x[c](), 'm3')
print()
print('Volume = ', sum(m.x[c]() for c in m.C), 'm3')
print('Cost = ', m.obj(),' €')

Optimal Blend
   A : 37.5 m3
   B : 62.5 m3
   C : 0.0 m3

Volume =  100.0 m3
Cost =  27.625  €


Además, el objeto `results` contiene información de interés sobre la solución obtenida que puede resultar útil para detectar errores.

In [36]:
print('Solver status: ' ,results.solver.status)
print('Termination condition: ', results.solver.termination_condition)

Solver status:  ok
Termination condition:  optimal
